## Quantifying a Spectrum

#### Note:  This notebook uses the modern (albeit slightly less flexible) `reference` and `references` functions to build the fitting standards.

This script show how to pull together the functionality in the NeXL X-ray microanalysis library to quantify a electron-excited X-ray spectrum using standard spectra.

First, we need to load the necessary libraries - `NeXLSpectrum` and `NeXLMatrixCorrection` from NeXL, `Gadfly` for plotting and `DataFrames` for tabulation. 

In [ ]:
using NeXLSpectrum
using NeXLMatrixCorrection
using Gadfly
using DataFrames

# Configure the environment to display larger tables
ENV["LINES"] = 60; ENV["COLUMNS"]=200

Next we specify where the spectra are located and load the spectrum from a file.  `loadspectrum(...)` can sniff the format of a spectrum file and load 'ISO/EMSA', 'ASPEX TIFF' and 'Bruker SPX' files automatically.

Once the spectrum is loaded into `unk`, we use the data in `unk` to build a suitable detector with a FWHM at Mn K$\alpha$ of 132.0 eV and a low-level discriminator cut-off of 120 channels.  Finally, we set the `:Detector` property of the spectrum to `det`.

Finally, we use `Gadfly`, which `NeXLSpectrum` has specialized to understand `Spectrum` structures, to plot the spectrum with KLM markers from 0.0 eV to 8,000 eV.

In [ ]:
path = joinpath(@__DIR__,"K309")
# Read the spectrum from disk (an ISO/EMSA file or other)
unk = loadspectrum(joinpath(path,"K309.msa"))
# Build a basic detector that matches the unknown spectrum 
det = matching(unk, 132.0, 120)
unk[:Detector] = det

# Configure an plot the unknown spectrum
set_default_plot_size(10inch,2.8inch)
plot(unk, klms=[n"O", n"Al", n"Si", n"Ca", n"Fe", n"Ba"], xmax=8.0e3)

Next, we define and load the reference spectra.  Here we use `n".."` notation to parse the element symbol and `mat"..."` to parse the composition.

In [ ]:
# Specify, load, filter the reference spectra
refs = references( [ 
    # reference( element, filename, material )
    reference(n"O", joinpath(path,"Al2O3 std.msa"), mat"Al2O3" ),
    reference(n"Al", joinpath(path,"Al2O3 std.msa"), mat"Al2O3" ),
    reference(n"Si", joinpath(path,"Si std.msa"), mat"Si" ),
    reference(n"Ca", joinpath(path,"CaF2 std.msa"), mat"CaF2" ),
    reference(n"Fe", joinpath(path,"Fe std.msa"), mat"Fe" ),
    reference(n"Ba", joinpath(path,"BaF2 std.msa"), mat"BaF2" )
], det)
# Plot the reference spectra
plot(refs, xmax = 8.0e3)

Finally, we apply the filter to the unknown and fit the filtered references.

In [ ]:
# Fit the unknown spectrum with the filtered references
res=fit_spectrum(unk, refs)
# Tabulate the results
asa(DataFrame,res)

To visualize these results, `Gadfly` has been specialized to plot `FilterFitResult` objects like this.  The plot shows the range of channels involved in each fitted ROI along with the associated k-ratio.  The red line is the filter fit residual.  It shows which x-rays were not accounted for as characteristic X-rays in one of the references.  You can use the residual to determine if any elements have been overlooked.

You can see that some of the ROIs are narrow (like the O K) while others are broad (like the Ba L). You can also see which ROIs interfere with each other (like the O K and Fe L or the Al K and Si K).

In [ ]:
plot(res)

The final step is to matrix correct the k-ratios to estimate the composition.

In [ ]:
# Takes the k-ratios in `res` and using the XPP algorithm, iterate to estimate the composition
iterres = quantify(res, iteration = Iteration(mc=XPP, fc=ReedFluorescence, cc=NoCoating))
# Display the results with matrix correction factors
asa(DataFrame, iterres)

In [ ]:
# Define the `nominal` composition for comparison
nominal = material("nominal", Dict(n"O"=>0.3872, n"Al"=>0.0794, n"Si"=>0.187, n"Ca"=>0.1072, n"Fe"=>0.1049, n"Ba"=>0.1343))
# Tabulate
asa(DataFrame, [ iterres.comp, nominal ])

Create a table that compares the measured and nominal composition.

In [ ]:
compare( iterres.comp, nominal)

NWMR 16-Sep-2021